In [ ]:
# If necessary:
import os
os.remove('snorkel.db')

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

from snorkel import SnorkelSession
session = SnorkelSession()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from snorkel.parser import CorpusParser
from snorkel.parser import HTMLParser
from snorkel.parser import OmniParser

docs_path = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware100_html/'
doc_parser = HTMLParser(path=docs_path)
context_parser = OmniParser()
cp = CorpusParser(doc_parser, context_parser, max_docs=2) 

In [ ]:
%time corpus = cp.parse_corpus(name='Hardware', session=session)

In [ ]:
session.add(corpus)
session.commit()

In [ ]:
#If necessary:
# import os
# os.system('cp snorkel.db snorkel.db\ corpus');

In [1]:
#If necessary:
import os
os.remove('snorkel.db');
os.system('cp snorkel.db\ corpus snorkel.db');

In [4]:
from snorkel.models import Corpus
from snorkel.utils import get_ORM_instance

corpus = get_ORM_instance(Corpus, session, 'Hardware')
print "%s contains %d Documents" % (corpus, len(corpus))

Corpus (Hardware) contains 10 Documents


In [5]:
import itertools
from collections import defaultdict
import numpy as np
from pdb import set_trace as t

total = 0
bad = 0
for doc in corpus.documents:
    for table in doc.tables:
        total += 1
        m = max(cell.row_end for cell in table.cells) + 1
        n = max(cell.col_end for cell in table.cells) + 1
        table_grid = np.zeros((m,n))
        for cell in table.cells:
            for i, j in itertools.product(
                range(cell.row_start, cell.row_end + 1), 
                range(cell.col_start, cell.col_end + 1)):
                table_grid[i,j] += 1
        for i, j in itertools.product(range(m), range(n)):
            if table_grid[(i,j)] != 1:
                bad += 1
#                 t()
                break
good = total - bad
print good, bad, total

93 0 93


Original test:

418 382 800

New test:

790 10 800